In [8]:
import networkx as nx
from utils import MatrixIO, FileUtils
import re, sys, json, os, glob
from operator import itemgetter
import numpy as np
import pandas as pd
mfio = MatrixIO()

In [9]:
def generate_prefix_list(node_list):
    '''Condition can be a lambda function, e.g. lambda xparts: True if xparts[3] != "bio2rdf.org" else False'''
    pref_types = {}
    for node in node_list:
        nparts = re.split("[#:/]", node)
        prefix = node[:len(node)-len(nparts[len(nparts)-1])]
        if not prefix in pref_types: pref_types[prefix] = []
        pref_types[prefix].append(node)
    return pref_types

first_cap_re = re.compile('(.)([A-Z][a-z]+)')
all_cap_re = re.compile('([a-z0-9])([A-Z])')

def parse_camel_case(name):
    s1 = first_cap_re.sub(r'\1_\2', name)
    return all_cap_re.sub(r'\1_\2', s1).lower()

def parse_uri_token(token):
    prop = parse_camel_case(token)
    pparts = re.split("[-_]", prop)
    name = " ".join([x.title() for x in pparts])
    return name

def parseURI(uri):
    nparts = re.split("[#:/]", uri)
    name = parse_uri_token(nparts[len(nparts)-1])
    return name

In [10]:
def _fileset(folder_path, exclusion_criteria=None):
    "A class of navigators on the file system"
    fileset = []
    fpath = "/Users/mkamdar/Desktop/PhD/lod_query/"
    for root, dirs, files in os.walk(folder_path):
        for x in dirs:
            _f = glob.glob(fpath + folder_path + x + "/str_inst_df.tsv") #*_inst_df.tsv
            fileset.extend(_f)
    return fileset

def get_samp_insts(f):
    br = pd.read_csv(f, sep="\t")
    brir = br[br["dtype"] == "iri"]
    samp_insts = list(brir["samp_insts"])
    #print samp_insts
    act = []
    for k in samp_insts:
        act.extend(eval(k))
    return act

#---------------------------------------------

In [ ]:
ontoSet = mfio.load_matrix("LDschemaontoSet.dat")
vocab_elem = mfio.load_matrix("vocab_elem.dat")

In [ ]:
fu = FileUtils()
mfio = MatrixIO()
dirn = ["inst_exp/", "inst_exp2/", "inst_exp3/"]
dsetin = {}
for m in dirn:
    fsets = _fileset(m)
    for k in fsets:
        dnamep = k.split("/")
        print k, dnamep[len(dnamep)-2]
        act = get_samp_insts(k)
        dsetin[dnamep[len(dnamep)-2]] = act

mfio.save_matrix(dsetin, "dsetin.dat")

In [ ]:
totin = []
for k in dsetin:
    for m in dsetin[k]:
        totin.append(m)
int_preflist = generate_prefix_list(totin)
sortint_preflist = sorted({k: len(int_preflist[k]) for k in int_preflist}.items(), key=itemgetter(1), reverse=True)
for k in sortint_preflist: print k

In [ ]:
for k in int_preflist:
    if k in disAllPrefs: print k, len(int_preflist[k])

In [ ]:
mfio.save_matrix(sortint_preflist, "sortint_preflist.dat")

#---------------------------------------------

In [ ]:
compSet = ontoSet
for m in lpreflist: 
    compSet[m] = {"dsets": set([]), "clcount": len(lpreflist[m]), 'type': 'unpublishedVocab'}
    for k in lpreflist[m]:
        if not redG.has_node(k) and k in dsetids:
            _dset = dsetids[k]
            compSet[m]["dsets"] = compSet[m]["dsets"].union(_dset)
        elif redG.has_node(k):
            compSet[m]["dsets"] = compSet[m]["dsets"].union(set([a.lower() for a in redG.node[k]["dsets"].split(":-:")]))
        elif combG.has_node(k):
            compSet[m]["dsets"] = compSet[m]["dsets"].union(set([a.lower() for a in combG.node[k]["dsets"].split(":-:")]))
print len(compSet)

In [ ]:
for k in compSet:
    compSet[k]["dsets"] = len(compSet[k]["dsets"])
    print k, compSet[k]

In [ ]:
lov_df_con = pd.read_csv("allVocab_terms.csv", quotechar='"', skipinitialspace=True)
tot_size = lambda x: lov_df_con[lov_df_con[x[0]].apply(lambda m: True if x[1][0:len(x[1])-1] in m else False)].shape
tot_size(("pURI ", 'uberon'))
tot_size_dict = {}
for k in vocab_elem.keys(): tot_size_dict[k.strip()] = tot_size(("vocab ", k))
print len(tot_size_dict)

In [ ]:
compSet_df = pd.DataFrame.from_dict(compSet, orient="index")
compSet_df = compSet_df.reset_index()
compSet_df = compSet_df.sort_values("clcount", ascending=False)
compSet_df.head()

In [ ]:
compSet_df.to_csv("compSet_df_m.tsv", sep="\t", index=None)

#---------------------------------------------

In [ ]:
dset_list = {}
dset_count = {}
for k in redG.nodes():
    if redG.node[k]["type"] == "ontoNode": continue
    dsets = set(redG.node[k]["dsets"].split(":-:")) - set([""])
    if not len(dsets) in dset_count: dset_count[len(dsets)] = set([])
    dset_count[len(dsets)].add(k)
    for m in dsets:
        if not m.lower() in dset_list: dset_list[m.lower()] = set([])
        dset_list[m.lower()].add(k)

In [ ]:
_dset_count = {x: len(dset_count[x]) for x in dset_count}
_dset_count = pd.DataFrame.from_dict(_dset_count, orient="index")
_dset_count = _dset_count.reset_index()
_dset_count.columns = ["dsetnumber", "sharedschema"]
_dset_count.to_csv("plot_dsethist.tsv", sep = "\t", index=None)

In [ ]:
top_sort = []
for k in dset_count:
    if k > 10: 
        for m in dset_count[k]: top_sort.append(m)

ap = generate_prefix_list(top_sort)
sortint_ap = sorted({k: len(ap[k]) for k in ap}.items(), key=itemgetter(1), reverse=True)
for k in sortint_ap: print k[0] + '\t' + str(k[1])